# Exercise
1. Rotate image by 45 degrees without cropping the sides of the image. (Hint: There are 2 strategies to tackle these problems). Use _"lena.jfif"_ as the input image.
    - Use external libraries `imutils`.  
    - Modify the transformation matrix.
2. Use the images with titles: _"flower.jfif"_ and _"native-bee.png"_. I want to put flower above an image. If I add two images, it will change color. If I blend it, I get a transparent effect. But I want it to be opaque. If it was a rectangular region, we could use the ROI as we did in the previous section. But flower is not a rectangular region. This is where bitwise operations, like AND, OR, NOT and XOR really come in handy. The associated functions are `cv.bitwise_and()`, `cv.bitwise_or()` and `cv.bitwise_not()`. You need to use `cv.threshold` function to segment the flower. Please refer to [online documentation](https://docs.opencv.org/4.x/d0/d86/tutorial_py_image_arithmetics.html) for more info. The result should resemble the following:  
![bee and flowers](img_embed/activity3.PNG "bee_flower")
3. Write a function that randomly crop the central region of an image. The method signature should be as shown in the following:
```
random_center_crop(image, min_crop_ratio, max_crop_ratio)
```

4. Aside from Gaussian noise, name another common type of noise. Write the code to demonstrate how the noise can be included in an image.

In [4]:
import cv2 as cv 
import numpy as np
import matplotlib.pyplot as plt
from utils import display_image, display_images

# Question 1
1. Rotate image by 45 degrees without cropping the sides of the image. (Hint: There are 2 strategies to tackle these problems). Use _"lena.jfif"_ as the input image.
    - Use external libraries `imutils`.  
    - Modify the transformation matrix.

In [8]:
!pip install imutils

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25854 sha256=88c9b2a142e3b5871136ece97682fc4c72fd942aa1df2a18e9a9bd85addb30a9
  Stored in directory: c:\users\asus\appdata\local\pip\cache\wheels\c2\02\32\f3617a9f68bcc67eda3ebeb4514eba18f62e81ff439428109d
Successfully built imutils


In [9]:
import imutils

In [11]:
img = cv.imread("images/lena.jfif")

height, width = img.shape[:2]
center = (width // 2, height // 2)
angle = 90
scale = 1
M = cv.getRotationMatrix2D(center, angle, scale)
dst = cv.warpAffine(img, M, (width, height))

display_images([img, dst], ("original", "rotation"))

In [7]:
M

array([[ 6.12323400e-17,  1.00000000e+00, -1.42108547e-14],
       [-1.00000000e+00,  6.12323400e-17,  2.24000000e+02]])

In [14]:
dst1 = imutils.rotate(img, angle, center, scale)
display_images([img, dst1], ("original", "rotation"))

# Question 2
2. Use the images with titles: _"flower.jfif"_ and _"native-bee.png"_. I want to put flower above an image. If I add two images, it will change color. If I blend it, I get a transparent effect. But I want it to be opaque. If it was a rectangular region, we could use the ROI as we did in the previous section. But flower is not a rectangular region. This is where bitwise operations, like AND, OR, NOT and XOR really come in handy. The associated functions are `cv.bitwise_and()`, `cv.bitwise_or()` and `cv.bitwise_not()`. You need to use `cv.threshold` function to segment the flower. Please refer to [online documentation](https://docs.opencv.org/4.x/d0/d86/tutorial_py_image_arithmetics.html) for more info. The result should resemble the following:  
![bee and flowers](img_embed/activity3.PNG "bee_flower")

In [5]:
img1 = cv.imread("images/flower.jfif")
img2 = cv.imread("images/native-bee.png")

# I want to put logo on top-left corner, So I create a ROI
rows, cols, channels = img1.shape
roi = img2[0:rows, 0:cols]

# Now create a mask of logo and create its inverse mask also
img1gray = cv.cvtColor(img1,cv.COLOR_BGR2GRAY)
ret, mask = cv.threshold(img1gray, 72, 255, cv.THRESH_BINARY)
mask_inv = cv.bitwise_not(mask)

# Now black-out the area of logo in ROI
img2_bg = cv.bitwise_and(roi,roi,mask = mask_inv)

# Take only region of logo from logo image.
img1_fg = cv.bitwise_and(img1,img1,mask = mask)

# Put logo in ROI and modify the main image
dst = cv.add(img2_bg,img1_fg)
img2[0:rows, 0:cols ] = dst

cv.imshow('res',img2)
cv.waitKey(0)
cv.destroyAllWindows()

# Question 3
3. Write a function that randomly crop the central region of an image. The method signature should be as shown in the following:
```
random_center_crop(image, min_crop_ratio, max_crop_ratio)
```


In [5]:
import random

In [14]:
def random_center_crop(image, min_crop_ratio, max_crop_ratio):
    # Get the image height and width
    height, width = image.shape[:2]

    # Ensure the crop ratios are between 0 and 1
    assert 0 < min_crop_ratio <= max_crop_ratio <= 1, "Crop ratios should be between 0 and 1"

    # Randomly determine the crop ratio
    crop_ratio = np.random.uniform(min_crop_ratio, max_crop_ratio)

    # Calculate the size of the crop
    crop_height = int(height * crop_ratio)
    crop_width = int(width * crop_ratio)

    # Calculate the center of the image
    center_y, center_x = height // 2, width // 2

    # Calculate the top-left corner of the crop
    start_y = max(0, center_y - crop_height // 2)
    start_x = max(0, center_x - crop_width // 2)

    # Calculate the bottom-right corner of the crop
    end_y = min(height, start_y + crop_height)
    end_x = min(width, start_x + crop_width)

    # Crop the image
    cropped_image = image[start_y:end_y, start_x:end_x]

    return cropped_image

In [15]:
# Example usage:
# Load an image

image = cv.imread("images/chessboard.png")

# Perform a random center crop
cropped_image = random_center_crop(image, 0.5, 0.8)

# Display the original and cropped images
cv.imshow("Original", image)
cv.imshow("Cropped", cropped_image)
cv.waitKey(0)
cv.destroyAllWindows()


In [ ]:
def random_center_crop(image, min_crop_ratio, max_crop_ratio):
    height, width = image.shape[:2]
    center = (width // 2, height // 2)

# Question 4
4. Aside from Gaussian noise, name another common type of noise. Write the code to demonstrate how the noise can be included in an image.

In [28]:
# Salt and pepper Noise
def add_salt_and_pepper_noise(img, salt_prob, pepper_prob):
    """"""
    noisy_image = img.copy()
    # add salt noise
    salt_mask = np.random.rand(*img.shape) < salt_prob
    noisy_image[salt_mask] = 255
    
    # add pepper noise
    pepper_mask = np.random.rand(*img.shape) < pepper_prob
    noisy_image[pepper_mask] = 0
    
    return noisy_image

In [29]:
img = cv.imread("images/room.jpg")
noise = add_salt_and_pepper_noise(img, 0.01, 0.01) # sigma increase, more noise 
   
display_images([img, noise], ("original", "noise"))